In [1]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
from collections import deque 
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import plotly.express as px

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [6]:
X,y = make_classification(15_000,500,random_state=50,weights=[0.9])
X, X_validate, y, y_validate = train_test_split(X,y, random_state=50)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =  0.7)

In [292]:
X_train_ = X_train.copy()
y_train_ = y_train.copy()
rng_idx = pd.Series(y_train).sample(len(y_train)).index
X_train_ = X_train_[rng_idx]
y_train_ = y_train_[rng_idx]
M_idx, m_idx = pd.DataFrame(y_train_).groupby(0).apply(lambda x: x.index.values)

In [230]:
# pd.Series(y_train).value_counts()   

In [231]:
# bal_idx = pd.DataFrame(y_train, columns = ['y']).groupby('y').apply(lambda x: x.sample(250)).index.to_frame()[1].values


In [232]:
# rng = np.random.default_rng(1235)

In [233]:
# rng.shuffle(bal_idx)

In [234]:
# X_bal = X_train[bal_idx]
# y_bal = y_train[bal_idx]

In [235]:
# X_bal = X_train.copy()
# y_bal = y_train.copy()

In [236]:
# M_idx, m_idx = pd.DataFrame(y_bal).groupby(0).apply(lambda x: x.index.values)

---

In [8]:
 actions = {0 : 0, 1 : 1}
 len(actions)

2

In [9]:
%run -i ./DQN_Class.py
gamma=0.5
epsilon=.8
numberEpisodes= 120
LearningQDeep=DeepQLearning(gamma,epsilon,numberEpisodes)


# NN approach

In [10]:
nn = LearningQDeep.buildNetwork()

In [11]:
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 128)               64128     
                                                                 
 dense_9 (Dense)             (None, 56)                7224      
                                                                 
 dense_10 (Dense)            (None, 1)                 57        
                                                                 
Total params: 71,409
Trainable params: 71,409
Non-trainable params: 0
_________________________________________________________________


In [12]:
early_stopping = tf.keras.callbacks.EarlyStopping(
                                monitor='val_precision', 
                                verbose=1,
                                patience=10,
                                mode='max',
                                restore_best_weights=True)

In [13]:
nn.fit(X_bal,y_bal, validation_data=[X_test,y_test], epochs=50, callbacks = [early_stopping])

NameError: name 'X_bal' is not defined

In [85]:
y_pred = [1 if x>0.5 else 0 for x in nn.predict(X_bal)]

16/16 [==============================] - 0s 2ms/step


In [86]:
print(confusion_matrix(y_bal, y_pred))

[[183  67]
 [ 14 236]]


In [87]:
print(classification_report(y_bal, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.73      0.82       250
           1       0.78      0.94      0.85       250

    accuracy                           0.84       500
   macro avg       0.85      0.84      0.84       500
weighted avg       0.85      0.84      0.84       500



In [29]:
y_pred = [1 if x>0.5 else 0 for x in nn.predict(X_test)]

247/247 [==============================] - 0s 1ms/step


In [83]:
print(confusion_matrix(y_test, y_pred))

[[1814 5225]
 [  88  748]]


In [84]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.26      0.41      7039
           1       0.13      0.89      0.22       836

    accuracy                           0.33      7875
   macro avg       0.54      0.58      0.31      7875
weighted avg       0.87      0.33      0.39      7875



---

In [14]:
def reward(si, a, M, m, y):
    l = y[si]

    if (a == l) & (si in m):
        r = 1
    elif  (a != l) & (si in m):
        r = -1
    if (a == l) & (si in M):
        r = 0.5
    elif  (a != l) & (si in M):
        r = -0.5
    return r

In [15]:
def step(action, i, M, m, X, y):

    r = reward(i,action, M, m, y)

    nextState = X[i+1,:]

    terminated = False
    
    l = y[i]

    if i == len(y) - 1:
        terminated = True
    elif (action != l) & (i in m):
        print(action, l)
        terminated = True


    return r, nextState, terminated, i+1
    

In [20]:
%run -i ./DQN_Class.py
gamma=0.2
epsilon=.9
numberEpisodes= 500
LearningQDeep=DeepQLearning(gamma,epsilon,numberEpisodes)


In [21]:
0.96**(500-25)*(0.9)

3.4125435562551007e-09

In [22]:
LearningQDeep.actionDimension

2

In [23]:
LearningQDeep.epsilon

0.9

In [24]:
LearningQDeep.mainNetwork.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 30)                15030     
                                                                 
 dense_20 (Dense)            (None, 64)                1984      
                                                                 
 dense_21 (Dense)            (None, 20)                1300      
                                                                 
 dense_22 (Dense)            (None, 2)                 42        
                                                                 
Total params: 18,356
Trainable params: 18,356
Non-trainable params: 0
_________________________________________________________________


In [25]:
%%time
LearningQDeep.trainingEpisodes()

Simulating episode 0
	 Fist train of main network...
		 [UPDATED] Target network
		 Current epsilon: 0.9
0 1
	 First 1 index: [ 2 21 47 51 58 69 70 74 78 81]
	 Reached sample: 22
	 Sum of rewards 0.0
Simulating episode 1
	 Fist train of main network...
0 1
	 First 1 index: [  5   9  25  58  61  68  69  98 113 117]
	 Reached sample: 10
	 Sum of rewards 0.0
Simulating episode 2
	 Fist train of main network...
		 [UPDATED] Target network
		 Current epsilon: 0.9
0 1
	 First 1 index: [ 19  43  49  54  59  70  72  84  89 101]
	 Reached sample: 20
	 Sum of rewards -0.5
Simulating episode 3
	 Fist train of main network...
		 [UPDATED] Target network
		 Current epsilon: 0.9
		 [UPDATED] Target network
		 Current epsilon: 0.9
		 [UPDATED] Target network
		 Current epsilon: 0.9
0 1
	 First 1 index: [ 16  26  34  49  57 124 132 143 162 177]
	 Reached sample: 58
	 Sum of rewards 1.5
Simulating episode 4
	 Fist train of main network...
		 [UPDATED] Target network
		 Current epsilon: 0.9
0 1
	 First 

In [26]:
loaded_model = LearningQDeep.mainNetwork

In [27]:
y_pred  = np.argmax(loaded_model.predict(X_train,verbose=0), axis = 1)

In [28]:
print(confusion_matrix(y_train, y_pred))

[[2988   68]
 [  56  263]]


In [29]:
print(classification_report(y_train,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      3056
           1       0.79      0.82      0.81       319

    accuracy                           0.96      3375
   macro avg       0.89      0.90      0.89      3375
weighted avg       0.96      0.96      0.96      3375



In [30]:
y_pred  = np.argmax(loaded_model.predict(X_test,verbose=0), axis = 1)

In [31]:
print(confusion_matrix(y_test, y_pred))

[[6761  259]
 [ 438  417]]


In [32]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      7020
           1       0.62      0.49      0.54       855

    accuracy                           0.91      7875
   macro avg       0.78      0.73      0.75      7875
weighted avg       0.90      0.91      0.91      7875



In [33]:
y_pred  = np.argmax(loaded_model.predict(X_validate,verbose=0), axis = 1)

In [34]:
print(confusion_matrix(y_validate, y_pred))

[[3250  123]
 [ 194  183]]


In [35]:
print(classification_report(y_validate,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      3373
           1       0.60      0.49      0.54       377

    accuracy                           0.92      3750
   macro avg       0.77      0.72      0.74      3750
weighted avg       0.91      0.92      0.91      3750



In [36]:
LearningQDeep.mainNetwork.save('Trained_model.h5')